Written by Zain Eris Kamal (zain.eris.kamal@rutgers.edu) on May 3, 2023.

Context:
- There's currently a satellite orbiting Mars called MAVEN, which has a magnetometer that measures magnetic fields. In order for this magnetometer to accurately measure the magnetic field of the crust, the satellite must be on the "night side" or "shadow" of the planet (approximately, see magnetotail studies). This is because highly energetic particles ejected from the Sun (called "solar wind") can create a plasma sheath around the planet that can interfere with the magnetic field measurements. 
    - Sample animation here: https://en.wikipedia.org/wiki/File:Animati3.gif

Problem:
- Currently, we filter "nightside" data by only looking at data collected between 20:00—09:00 according to the satellite's clock. However, this is in UTC, not Mars time. Lol.

Solution:
- Using the positions of the Sun, Mars, and MAVEN, we can geometrically determine when MAVEN is in the "shadow" of Mars. Another way to think about this is if you shoot a ray from the Sun to MAVEN, does it intersect Mars?

Conclusion:
- I use VPython to animate a simple star/planet/satellite scenario, which validates whether my vector math is correct. 
- It works: https://files.catbox.moe/7lngm4.mp4

Future:
- Use this vector math on actual MAVEN data. Maybe double-check with VPython.
- Should I do these calculations with VPython or numpy? VPython provides really strong vector object and operations. See bottom-most cells for how I've implemented some of these operations in simple math.

---

In [32]:
import numpy as np

In [72]:
from vpython import *
scene = canvas()



# # scene = canvas()
# sphere()





# Calculate the force vector exerted on b1 by b2
def f_g(b1, b2):
    if b1.pos == b2.pos:
        return vec(0, 0, 0) # The force of a body on itself is 0

    G = 1 # Use 6.67430E-11 for real world values; 1 is more convenient
    
    r_vec = b2.pos - b1.pos # Vector from b1 to b2

    # Prevent gravitational force becoming absurdly large when bodies pass through each other
    if mag(r_vec) < (b1.radius + b2.radius):
        r_mag = b1.radius + b2.radius
    else:
        r_mag = mag(r_vec)

    return ((G * b1.mass * b2.mass) / (r_mag ** 2)) * hat(r_vec) # Equation for gravity force vector






sun = sphere(
    pos=vec(-10,-7,-4),
    color=vec(1,1,0),
)

mars = sphere(
    pos=vec(0,0,0),
    color=vec(1,0.5,0),
    radius=0.5,
    mass=100
)

maven = sphere(
    pos = mars.pos + vec(1,-1,1),
    radius=0.1,
    mass=1,
    v=vec(3,6,-1),
    color=vec(1,0,0)
)


arr = arrow(
    pos=sun.pos,
    axis=maven.pos-sun.pos,
    shaftwidth=0.1,
    headwidth=0.2,
    color=vec(0,1,0)
)



''' what angle does the shadow of mars make? i.e. what's the angle between sun2mars and a vector to a point on the surface of Mars that lies on the plane perpendicular to sun2mars. '''



sun2mars = mars.pos-sun.pos
sun2mvn = maven.pos-sun.pos



mars2surface = norm( vec(-sun2mars.y, sun2mars.x, 0) ) * mars.radius

half_angle_mars = diff_angle(sun2mars, mars2surface-sun.pos)


## ^ check this with:
# print(dot(mars2surface, sun2mars))








''' animation '''




t=0
dt = 0.01
speed = 1 / (5 * dt)

while t < 1000:
    rate(speed)
    
    ''' calculate intersection '''
    
    sun2mars = mars.pos-sun.pos
    sun2mvn = maven.pos-sun.pos
    
    curr_angle = diff_angle(sun2mars, sun2mvn)
    
    if ( (curr_angle < half_angle_mars) and (sun2mars.mag < sun2mvn.mag) ):
        arr.color = vec(1,0,0)
    else:
        arr.color = vec(0,1,0)
    
    
    
    
    ''' animate '''
    
    maven.v += (f_g(maven,mars)/maven.mass)*dt
    maven.pos += maven.v*dt
    
    
    arr.axis = sun2mvn
    
    t+=dt

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [75]:
# print(sun2mars)
# print(sun2mvn)
# print()


# def dot(a: vector, b: vector) -> float:
#     return a.x*b.x + a.y*b.y + a.z*b.z
# print(dot(sun2mars,sun2mvn))


# print(sun2mars.dot(sun2mvn))
# print()

# print(sun2mars.mag)
# print(sun2mvn.mag)
# print()


''' Hard-coding the `diff_angle` function in case I have to implement this in numpy '''

print(np.arccos(
    sun2mars.dot(sun2mvn) / (sun2mars.mag * sun2mvn.mag)
))

print(diff_angle(sun2mars, sun2mvn))

# print(half_angle_mars)

0.13968322270977082
0.13968322270977002


In [76]:
''' Verifying that my `half_angle_mars` is correct '''
print(dot(mars2surface, sun2mars))

0.0
